In [ ]:
import pandas as pd
import altair as alt
import sys
sys.path.append('figure_scripts/')
import plot_scripts as ps

In [ ]:
success_table = pd.read_csv('17_newly_diagnosed_results.tsv', sep='\t')
success_table

id_list = list(set(success_table['UDN_ID']))

print(len(set(success_table['UDN_ID'])), 'probands')
print(len(set(success_table['ID'])), 'genes')
print(len(success_table), 'variants')

In [ ]:
mapping = pd.read_csv('GS_ID_mapping.csv')
mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [ ]:
RUN_TYPES = ['exomiser_default', 'filtered_exomiser_default','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL','noN_filtered_15_85_human_revel_mvp_noWL']

In [ ]:
d_df,denom = ps.create_df(300, RUN_TYPES, success_table)
plot = ps.create_plot(denom, d_df, RUN_TYPES, 'set1', False, RUN_TYPES).properties(height=200, width=300)
plot

In [ ]:
order = list(success_table.sort_values(by='Variant_Level_noMOI_rank_noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL')['Dumb_ID'])
optimized = alt.Chart(success_table).mark_point(filled=True, size=100, opacity=1, stroke='black', strokeWidth=1).encode(
    x=alt.X('Dumb_ID', sort=order,axis=alt.Axis(labelAngle=-60)),
    y='Variant_Level_noMOI_rank_noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL',
    color=alt.Color('Exclude?',scale=alt.Scale(scheme='category10', domain=['','b','No']))
)

default_filtered = alt.Chart(success_table).mark_point(filled=True, size=100, opacity=1, stroke='black', strokeWidth=1).encode(
    x=alt.X('Dumb_ID', sort=order,axis=alt.Axis(labelAngle=-60)),
    y='Variant_Level_noMOI_rank_filtered_exomiser_default',
    color=alt.Color(scale=alt.Scale(scheme='category10'))
)

panel_b =alt.layer( default_filtered, optimized).properties(height=250, width=300).resolve_scale(color='independent')

In [ ]:
alt.hconcat(plot, panel_b).configure_legend(labelLimit=0).configure_axis(
        labelFontSize=13,
        titleFontSize=13).configure_legend(labelLimit=0,labelFontSize=13, titleFontSize=12)